# Extending the Generalized Sector

Table of Contents:

1. [Extending outputs](#Extending-outputs)
    1. [Simple output extension](#Simple-output-extension)
    1. [Adding TOML parameters to the outputs](#Adding-TOML-parameters-to-the-outputs)
1. [Modeling early obsolescence](#Modeling-early-obsolescence)
1. [Where to store new functionality](#Where-to-store-new-functionality)

One key feature of the generalized sector's implementation is that it should be easy to
extend. As such, most the steps shown in [Generalized Sector
Overview](GeneralizedSectorOverview.ipynb) can be made to run custom python functions,
as long as these inputs and output of the function follow a standard specific to each
step. We will look at a few here. Below is a list of possible hooks, referenced by their
implementation in the MUSE model itself

- `register_interaction_net` in `muse.interactions`: a list of lists of agents
  that do interact together.
- `register_agent_interaction` in `muse.interactions`: Given a list of
  interacting agents, perform the interaction itself.
- `register_production` in `muse.production`: A method to compute the production
  from a sector, given the demand and the capacity.
- `register_initial_asset_transform` in `muse.hooks`: Prior to investing,
  within an agent, allows any kind of transformation to be applied to the assets.
- `register_final_asset_transform` in `muse.hooks`: After computing investment,
  within an agent, sets the assets that will be owned the agents.
- `register_demand_share` in `muse.demand_share`: During agent investment, share
  of the demand that an agent will try and satisfy.
- `register_filter` in `muse.filters`: During agent investment, a
  filter to remove technologies from consideration.
- `register_objective` in `muse.objectives`: During agent investment, a
  quantity defining preference between one or another technology during investment.
- `register_decision` in `muse.decisions`: During agent investment, a transform
  to aggregate multiple objectives into a single one, e.g. via a weighted sum.
- `register_investment` in `muse.investment`: During agent investment, matches
  the demand to future investment using the decision metric above.
- `register_output_quantity` in `muse.output.sector`: A sectorial quantity to output for
  postmortem analysis.
- `register_output_sink` in `muse.outputs`: A _place_ where to store an output
  quantity, e.g. a file with a given format, a database on premise or on the cloud,
  etc...
- `register_carbon_budget_fitter` in `muse.carbon_budget`
- `register_carbon_budget_method` in `muse.carbon_budget`
- `register_sector`: registers a function that can create a sector from a muse
  configuration object.

## Extending outputs

MUSE can be told to save quantities for post-mortem analysis. There are actually two
steps to this process. First, we want to compute the quantity of interest. Second, we
want to store in a __sink__, somewhere. Generally, we can mix and match between
quantities and sinks; for instance we can compute the capacity and save it as csv file,
or an netcdf file.

### Simple output extension

To demonstrate this, we will compute a new quantity of (little) interest, and then save
it as is to a text file:

In [1]:
from xarray import Dataset, DataArray
from muse import outputs

@outputs.sector.register_output_quantity
def bracket(
    market: Dataset,
    capacity: DataArray,
    technologies: Dataset
) -> DataArray:
    from numpy import logical_and
    return logical_and(capacity > 10, capacity < 100)

The function we wrote takes three arguments. These are mandatory **for this hook**, the
hook which allows compute quantities for post-mortem analysis. We will see shortly other
hooks require other arguments. The function outputs the quantity that we want to save.

Importantly, the function is _decorated_: on the line above it sits the decorator
`@register_output_quantity`. This decorator ensures the new quantity will be addressable
in the TOML file.

Now we can write a sink. In this case, the sink will simply dump the quantity it is
given to a file, with a small message:

In [2]:
from typing import Any, Text
from muse.outputs.sinks import register_output_sink, sink_to_file

@register_output_sink(name="txt")
@sink_to_file(".txt")
def text_dump(data: Any, filename: Text) -> None:
    from pathlib import Path
    Path(filename).write_text(f"Hello, world!\n\n{data}")

There code above makes use of two decorators. The first one we expect,
`@register_output_sink`. It registers the function with MUSE, so that the sink is
addressable from a TOML file. The second one, `@sink_to_file`, is optional. It adds to
sinks that are files some nice-to-have features, such as a way to specify filenames and
check that files cannot be overwritten, unless explicitly allowed to.

Now we want to modify the input file to actually use this output type. Specifically, we
add a section to the output table:


```toml
[[sectors.commercial.outputs]]
quantity = "bracket"
sink = "txt"
filename = "{cwd}/{default_output_dir}/{Sector}{Quantity}{year}{suffix}"
```


The last line above allows us to specify the name of the file. We could also use
`sector` (no caps) and `quantity`.

There can be as many sections of this kind as we want in the TOML file, allowing for
multiple outputs.

In the following, we  first copy the default model provided with muse to a local subfolder called "model". Then we read the `settings.toml` file and modify it using python. You may prefer to modify the `settings.toml` file using your favorite text editor. However, modifying the file programmatically allows us to
routinely run this notebook as part of MUSE's test suite and check that the tutorial it is still up
to date.

In [3]:
from pathlib import Path
from toml import load, dump
from muse import examples

model_path = examples.copy_model(overwrite=True)
settings = load(model_path / "settings.toml")
new_output = {
    "quantity": "bracket",
    "sink":  "txt",
    "overwrite": True,
    "filename": "{cwd}/{default_output_dir}/{Sector}{Quantity}{year}{suffix}",
}
settings["sectors"]["residential"]["outputs"].append(new_output)
dump(settings, (model_path / "modified_settings.toml").open("w"))
settings

{'time_framework': [2020, 2025, 2030, 2035, 2040, 2045, 2050],
 'foresight': 5,
 'regions': ['USA'],
 'interest_rate': 0.1,
 'interpolation_mode': 'Active',
 'log_level': 'info',
 'equilibrium_variable': 'demand',
 'maximum_iterations': 100,
 'tolerance': 0.1,
 'tolerance_unmet_demand': -0.1,
 'outputs': [{'quantity': 'prices',
   'sink': 'aggregate',
   'filename': '{cwd}/{default_output_dir}/MCA{Quantity}.csv'},
  {'quantity': 'capacity',
   'sink': 'aggregate',
   'filename': '{cwd}/{default_output_dir}/MCA{Quantity}.csv'}],
 'carbon_budget_control': {'budget': []},
 'global_input_files': {'projections': '{path}/input/Projections.csv',
  'global_commodities': '{path}/input/GlobalCommodities.csv'},
 'sectors': {'residential': {'type': 'default',
   'priority': 1,
   'dispatch_production': 'share',
   'technodata': '{path}/technodata/residential/Technodata.csv',
   'commodities_in': '{path}/technodata/residential/CommIn.csv',
   'commodities_out': '{path}/technodata/residential/CommOu

We can now try and run the simulation. There are two ways to do this. From the
command-line, we can do `python3 -m muse data/commercial/settings2.toml` (note
that slashes may be the other way on Windows). Directly from the notebook, we can do:

In [4]:
from muse.mca import MCA

mca = MCA.factory(model_path / "modified_settings.toml")
mca.run()

-- 2020-07-07 16:12:15 - muse.readers.toml - INFO
Reading MUSE settings

-- 2020-07-07 16:12:15 - muse.readers.toml - INFO
 Default input values used: carbon_budget_control.commodities, carbon_budget_control.method, carbon_budget_control.debug, carbon_budget_control.control_undershoot, carbon_budget_control.control_overshoot, carbon_budget_control.method_options

-- 2020-07-07 16:12:15 - muse.readers.toml - INFO
 Default input values used: global_input_files.path

-- 2020-07-07 16:12:15 - muse.readers.toml - INFO
 Default input values used: expect_equilibrium, excluded_commodities, plugins

-- 2020-07-07 16:12:15 - muse.readers.toml - INFO
 Validating input settings...

-- 2020-07-07 16:12:15 - muse.readers.toml - INFO
Computing settings_check: check_log_level

-- 2020-07-07 16:12:15 - muse.decorators - INFO
 check_log_level PASSED

-- 2020-07-07 16:12:15 - muse.readers.toml - INFO
Computing settings_check: check_interpolation_mode

-- 2020-07-07 16:12:15 - muse.decorators - INFO
 chec

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 149.1019498703      
0.3675729138193     0.3675729138193     0.3675729138194     0.6468967809453  0.3675729138193     897.7040172625      
0.1586603347581     0.1586603347581     0.1586603347581     0.6045622515835  0.1586603347581     1052.983832701      
0.00534123475523    0.005341234755232   0.005341234755235   0.9868065920797  0.005341234755234   219.6399956184      
0.00202447267962    0.00202447267962    0.002024472679622   0.6402354415555  0.002024472679621   184.1460808456      
0.001250110255268   0.001250110255268   0.001250110255269   0.4028712420146  0.001250110255269   243.9362311365      
0.0001937966400292  0.0001937966400293  0.0001937966400294  0.8586478166717  0.0001937966400299  307.3188749593      
3.969593492852e-07  3.969593492763e-07  3.96959349281e-07

-- 2020-07-07 16:12:18 - muse.investments - INFO
Computing investment: scipy

-- 2020-07-07 16:12:18 - muse.investments - INFO
Minimized normalized capacity constraints successfully. 

-- 2020-07-07 16:12:18 - muse.hooks - INFO
Computing final_asset_transform: merge

-- 2020-07-07 16:12:18 - muse.production - INFO
Computing production: share



Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 149.1019498703      
0.3672629696663     0.3672629696663     0.3672629696661     0.6472109427944  0.3672629696662     898.9605582583      
0.1591309507283     0.1591309507283     0.1591309507282     0.6028434191867  0.1591309507283     1058.312258395      
0.006645868145419   0.006645868145423   0.006645868145411   0.9786587496431  0.006645868145421   340.7456005081      
0.003247672307472   0.003247672307474   0.003247672307468   0.5496977506866  0.003247672307473   514.3270959421      
0.0003868091351776  0.0003868091351778  0.0003868091351771  0.8920747447949  0.0003868091351778  669.8515006431      
1.892424581431e-07  1.892424581398e-07  1.892424581621e-07  0.999550454676   1.892424581693e-07  673.4193641187      
9.462110702983e-12  9.462131203994e-12  9.462131143618e-1

-- 2020-07-07 16:12:19 - muse.sectors.sector - INFO
Running power for year 2020

-- 2020-07-07 16:12:19 - muse.interactions - INFO
Net new_to_retro of 1 interactions interacting via transfer

-- 2020-07-07 16:12:19 - muse.demand_share - INFO
Computing demand_share: default

-- 2020-07-07 16:12:19 - muse.production - INFO
Computing production: max

-- 2020-07-07 16:12:19 - muse.production - INFO
Computing production: max

-- 2020-07-07 16:12:19 - muse.hooks - INFO
Computing initial_asset_transform: default

-- 2020-07-07 16:12:19 - muse.filters - INFO
Computing initial search-space: from_techs

-- 2020-07-07 16:12:19 - muse.filters - INFO
Computing filter: compress

-- 2020-07-07 16:12:19 - muse.objectives - INFO
Computing objective: LCOE

-- 2020-07-07 16:12:19 - muse.decisions - INFO
Computing decision: single

-- 2020-07-07 16:12:19 - muse.investments - INFO
Computing investment: scipy

-- 2020-07-07 16:12:19 - muse.investments - INFO
Minimized normalized capacity constraints success

...

-- 2020-07-07 16:13:03 - muse.sectors.sector - INFO
Running power for year 2050

-- 2020-07-07 16:13:03 - muse.interactions - INFO
Net new_to_retro of 1 interactions interacting via transfer

-- 2020-07-07 16:13:03 - muse.demand_share - INFO
Computing demand_share: default

-- 2020-07-07 16:13:03 - muse.production - INFO
Computing production: max

-- 2020-07-07 16:13:03 - muse.production - INFO
Computing production: max

-- 2020-07-07 16:13:03 - muse.hooks - INFO
Computing initial_asset_transform: default

-- 2020-07-07 16:13:03 - muse.filters - INFO
Computing initial search-space: from_techs

-- 2020-07-07 16:13:03 - muse.filters - INFO
Computing filter: compress

-- 2020-07-07 16:13:03 - muse.objectives - INFO
Computing objective: LCOE

-- 2020-07-07 16:13:03 - muse.decisions - INFO
Computing decision: single

-- 2020-07-07 16:13:03 - muse.investments - INFO
Computing investment: scipy

-- 2020-07-07 16:13:04 - muse.investments - INFO
Minimized normalized capacity constraints success

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 599.4693530625      
0.03492386231018    0.03492386231017    0.03492386231025    0.9712098514233  0.03492386231018    1214.869087853      
0.007423235089608   0.007423235089607   0.007423235089624   0.8236890678862  0.007423235089608   540.8074092812      
0.0002087271516539  0.0002087271516282  0.0002087271516286  0.9803191650543  0.0002087271516282  12.52452762595      
2.87983687422e-05   2.879836873864e-05  2.879836873871e-05  0.8663922877472  2.879836873865e-05  1.777764275921      
1.464996266131e-07  1.464996400413e-07  1.464996400391e-07  1.0              1.464996400388e-07  0.006382806548431   
8.918617406667e-12  8.918644146627e-12  8.918642356452e-12  0.9999391217498  8.918642356457e-12  3.887887924001e-07  
1.386069411354e-12  1.38611425541e-12   1.386114077086e-1

-- 2020-07-07 16:13:04 - muse.investments - INFO
Computing investment: scipy

-- 2020-07-07 16:13:04 - muse.investments - INFO
Minimized normalized capacity constraints successfully. 

-- 2020-07-07 16:13:04 - muse.hooks - INFO
Computing final_asset_transform: merge

-- 2020-07-07 16:13:04 - muse.production - INFO
Computing production: share



Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 599.4693530625      
0.06042992940962    0.0604299294096     0.06042992940964    0.9459366582215  0.06042992940959    953.8509336505      
0.02295318750859    0.02295318750859    0.0229531875086     0.6655596202526  0.02295318750858    920.6610949816      
0.001138584502195   0.001138584502195   0.001138584502196   0.9986116272963  0.001138584502195   188.3248347854      
3.838725160114e-05  3.838725160114e-05  3.838725160116e-05  0.9675657301212  3.838725160113e-05  19.4018111094       
8.316521041187e-07  8.316521040978e-07  8.316521041008e-07  0.9906520730576  8.316521041001e-07  8.914458490867      
6.362068745172e-10  6.362068753701e-10  6.36206877078e-10   0.9992350087754  6.362068771031e-10  8.833832577065      
1.40412934035e-10   1.402175213906e-10  1.402175219445e-1

-- 2020-07-07 16:13:04 - muse.sectors.sector - INFO
Running gas for year 2050

-- 2020-07-07 16:13:04 - muse.interactions - INFO
Net new_to_retro of 1 interactions interacting via transfer

-- 2020-07-07 16:13:04 - muse.demand_share - INFO
Computing demand_share: default

-- 2020-07-07 16:13:04 - muse.production - INFO
Computing production: max

-- 2020-07-07 16:13:04 - muse.production - INFO
Computing production: max

-- 2020-07-07 16:13:05 - muse.hooks - INFO
Computing initial_asset_transform: default

-- 2020-07-07 16:13:05 - muse.hooks - INFO
Computing initial_asset_transform: clean

-- 2020-07-07 16:13:05 - muse.production - INFO
Computing production: share

-- 2020-07-07 16:13:05 - muse.mca - WARNING
Check growth constraints for wind.

-- 2020-07-07 16:13:05 - muse.mca - INFO
Finish simulation year 2050!



We can check that running the simulation has created the files we expect:

In [5]:
all_txt_files = sorted((Path() / "Results").glob("Residential*.txt"))
assert len(all_txt_files) == 7
assert "Hello, world!" in all_txt_files[0].read_text()
all_txt_files

[PosixPath('Results/ResidentialBracket2020.txt'),
 PosixPath('Results/ResidentialBracket2025.txt'),
 PosixPath('Results/ResidentialBracket2030.txt'),
 PosixPath('Results/ResidentialBracket2035.txt'),
 PosixPath('Results/ResidentialBracket2040.txt'),
 PosixPath('Results/ResidentialBracket2045.txt'),
 PosixPath('Results/ResidentialBracket2050.txt')]

### Adding TOML parameters to the outputs

The bracket quantity we are computing would be (somewhat) more useful if we could
specify the bracket range straight from the TOML file. Similarly, the "hello world"
message in the sink could also do with being parameterized. Not all hooks are this
flexible (for historical reasons, rather than any intrinsic difficulty). However, for
outputs, we can do this as follows:

In [6]:
from typing import Optional

@outputs.sector.register_output_quantity(overwrite=True)
def bracket(
    market: Dataset,
    capacity: DataArray,
    technologies: Dataset,
    lower: Optional[int] = 10,
    upper: Optional[int] = 100,
) -> DataArray:
    from numpy import logical_and
    return logical_and(capacity > lower, capacity < upper)


@register_output_sink(name="txt", overwrite=True)
@sink_to_file(".txt")
def text_dump(
    data: Any,
    filename: Text,
    msg : Optional[Text] = "Hello, world!"
) -> None:
    from pathlib import Path
    Path(filename).write_text(f"{msg}\n\n{data}")

We simply added parameters as keyword arguments to the functions.

Note: The `overwrite` argument allows us to overwrite previously defined registered
    functions. This is useful in a notebook such as this. But it should not be used in
    general. If `overwrite` were false, then the code would issue a warning and it would
    leave the TOML to refer to the original functions at the beginning of the notebook.
    This is useful to ensure custom modules do not silently overwrite each others registered functions.

Now we can modify the output section to take additional arguments:


```toml
[[sectors.commercial.outputs]]
quantity.name = "bracket"
quantity.upper = 10
quantity.lower = 1
sink.name = "txt"
sink.filename = "{cwd}/{default_output_dir}/{Sector}{Quantity}{year}{suffix}"
sink.msg = "Hello, you!"
sink.overwrite = True
```

Now, both `sink` and `quantity` are dictionaries which can take any number of arguments.
Previously, we were using a shorthand for convenience.

In [7]:
from toml import load, dump
from pathlib import Path

settings = load(model_path / "settings.toml")
settings["sectors"]["residential"]["outputs"] = [
    {
        "quantity": {
            "name": "bracket",
            "upper": 10,
            "lower": 1
        },
        "sink":  {
            "name": "txt", 
            "filename": "{cwd}/{default_output_dir}/{Sector}{Quantity}{year}{suffix}",
            "msg": "Hello, you!",
            "overwrite": True,
        }
    }
]
dump(settings, (model_path / "modified_settings.toml").open("w"))
settings

{'time_framework': [2020, 2025, 2030, 2035, 2040, 2045, 2050],
 'foresight': 5,
 'regions': ['USA'],
 'interest_rate': 0.1,
 'interpolation_mode': 'Active',
 'log_level': 'info',
 'equilibrium_variable': 'demand',
 'maximum_iterations': 100,
 'tolerance': 0.1,
 'tolerance_unmet_demand': -0.1,
 'outputs': [{'quantity': 'prices',
   'sink': 'aggregate',
   'filename': '{cwd}/{default_output_dir}/MCA{Quantity}.csv'},
  {'quantity': 'capacity',
   'sink': 'aggregate',
   'filename': '{cwd}/{default_output_dir}/MCA{Quantity}.csv'}],
 'carbon_budget_control': {'budget': []},
 'global_input_files': {'projections': '{path}/input/Projections.csv',
  'global_commodities': '{path}/input/GlobalCommodities.csv'},
 'sectors': {'residential': {'type': 'default',
   'priority': 1,
   'dispatch_production': 'share',
   'technodata': '{path}/technodata/residential/Technodata.csv',
   'commodities_in': '{path}/technodata/residential/CommIn.csv',
   'commodities_out': '{path}/technodata/residential/CommOu

We then run the simulation again:

In [8]:
import logging
from muse.mca import MCA
logging.getLogger("muse").setLevel(0)
mca = MCA.factory(model_path / "modified_settings.toml")
mca.run()

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 149.1019498703      
0.3675729138193     0.3675729138193     0.3675729138194     0.6468967809453  0.3675729138193     897.7040172625      
0.1586603347581     0.1586603347581     0.1586603347581     0.6045622515835  0.1586603347581     1052.983832701      
0.00534123475523    0.005341234755232   0.005341234755235   0.9868065920797  0.005341234755234   219.6399956184      
0.00202447267962    0.00202447267962    0.002024472679622   0.6402354415555  0.002024472679621   184.1460808456      
0.001250110255268   0.001250110255268   0.001250110255269   0.4028712420146  0.001250110255269   243.9362311365      
0.0001937966400292  0.0001937966400293  0.0001937966400294  0.8586478166717  0.0001937966400299  307.3188749593      
3.969593492852e-07  3.969593492763e-07  3.96959349281e-07

-- 2020-07-07 16:13:10 - muse.mca - WARNING
Check growth constraints for wind.



Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 149.1019498703      
0.3675729138193     0.3675729138193     0.3675729138194     0.6468967809453  0.3675729138193     897.7040172625      
0.1586603347581     0.1586603347581     0.1586603347581     0.6045622515835  0.1586603347581     1052.983832701      
0.00534123475523    0.005341234755232   0.005341234755235   0.9868065920797  0.005341234755234   219.6399956184      
0.00202447267962    0.00202447267962    0.002024472679622   0.6402354415555  0.002024472679621   184.1460808456      
0.001250110255268   0.001250110255268   0.001250110255269   0.4028712420146  0.001250110255269   243.9362311365      
0.0001937966400292  0.0001937966400293  0.0001937966400294  0.8586478166717  0.0001937966400299  307.3188749593      
3.969593492852e-07  3.969593492763e-07  3.96959349281e-07

-- 2020-07-07 16:13:15 - muse.mca - WARNING
Check growth constraints for wind.



Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 359.5446737952      
0.5430345896119     0.5430345896119     0.543034589612      0.4665449449403  0.5430345896119     1089.007211438      
0.2401687414292     0.2401687414292     0.2401687414292     0.5916998099899  0.2401687414292     1269.441899666      
0.007081390118295   0.007081390118287   0.007081390118282   1.0              0.007081390118286   277.4276509731      
0.003304944934454   0.003304944934451   0.003304944934448   0.5559974621066  0.00330494493445    218.3207978318      
0.001220657392565   0.001220657392563   0.001220657392562   0.6621916104328  0.001220657392563   334.2629101766      
0.0006385241089642  0.0006385241089635  0.000638524108963   0.4996315910358  0.0006385241089628  349.4700209526      
9.176691639401e-05  9.176691639391e-05  9.176691639385e-0

...

-- 2020-07-07 16:13:48 - muse.mca - WARNING
Check growth constraints for wind.



Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 647.7299898461      
0.2970631483991     0.2970631483991     0.2970631483992     0.713223582211   0.2970631483991     847.7676869428      
0.1783471451838     0.1783471451837     0.1783471451837     0.4239687712153  0.1783471451837     170.4131435796      
0.134953629791      0.1349536297909     0.1349536297909     0.2689797766954  0.1349536297909     539.2073432379      
0.03438177240922    0.03438177240921    0.03438177240921    0.7483797358084  0.0343817724092     442.4655943642      
0.01709429088645    0.01709429088644    0.01709429088644    0.5332319553822  0.01709429088644    862.6138580137      
0.001013118437332   0.001013118437324   0.001013118437324   0.9472732618992  0.001013118437439   1791.297656459      
4.459062157279e-07  4.459062157045e-07  4.4590621571e-07 

-- 2020-07-07 16:13:51 - muse.mca - WARNING
Check growth constraints for wind.



Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 647.7299898461      
0.2970631483991     0.2970631483991     0.2970631483992     0.713223582211   0.2970631483991     847.7676869428      
0.1783471451838     0.1783471451837     0.1783471451837     0.4239687712153  0.1783471451837     170.4131435796      
0.134953629791      0.1349536297909     0.1349536297909     0.2689797766954  0.1349536297909     539.2073432379      
0.03438177240922    0.03438177240921    0.03438177240921    0.7483797358084  0.0343817724092     442.4655943642      
0.01709429088645    0.01709429088644    0.01709429088644    0.5332319553822  0.01709429088644    862.6138580137      
0.001013118437332   0.001013118437324   0.001013118437324   0.9472732618992  0.001013118437439   1791.297656459      
4.459062157279e-07  4.459062157045e-07  4.4590621571e-07 

-- 2020-07-07 16:13:54 - muse.mca - WARNING
Check growth constraints for wind.



And we can check the parameters were used accordingly:

In [9]:
all_txt_files = sorted((Path() / "Results").glob("Residential*.txt"))
assert len(all_txt_files) == 7
assert "Hello, you!" in all_txt_files[0].read_text()
all_txt_files

[PosixPath('Results/ResidentialBracket2020.txt'),
 PosixPath('Results/ResidentialBracket2025.txt'),
 PosixPath('Results/ResidentialBracket2030.txt'),
 PosixPath('Results/ResidentialBracket2035.txt'),
 PosixPath('Results/ResidentialBracket2040.txt'),
 PosixPath('Results/ResidentialBracket2045.txt'),
 PosixPath('Results/ResidentialBracket2050.txt')]

## Where to store new functionality

As was demonstrated above, new functionality can be added easily. However, running a
jupyter notebook is not always the best approach. One can also store the functions in an
arbitrary python file:

In [11]:
%%writefile mynewfunctions.py
from muse.outputs.sinks import register_output_sink, sink_to_file

@register_output_sink(name="dummy")
@sink_to_file(".dummy")
def dummy(data, filename):
    from pathlib import Path
    Path(filename).write_text("Nothing to see")

Writing mynewfunctions.py


Then we can tell the TOML file where to find it:


```toml
plugins = "{cwd}/mynewfunctions.py"

[[sectors.commercial.outputs]]
quantity = "capacity"
sink = "dummy"
overwrite = true
```

Alternatively, `plugin` can also be given a list of paths rather than just a single one, as done below.

In [12]:
settings = load(model_path / "settings.toml")
settings["plugins"] = ["{cwd}/mynewfunctions.py"]
settings["sectors"]["residential"]["outputs"] = [
    {
        "quantity": "capacity",
        "sink":  "dummy",
        "overwrite": "true"
    }
]
dump(settings, (model_path / "modified_settings.toml").open("w"))
settings

{'time_framework': [2020, 2025, 2030, 2035, 2040, 2045, 2050],
 'foresight': 5,
 'regions': ['USA'],
 'interest_rate': 0.1,
 'interpolation_mode': 'Active',
 'log_level': 'info',
 'equilibrium_variable': 'demand',
 'maximum_iterations': 100,
 'tolerance': 0.1,
 'tolerance_unmet_demand': -0.1,
 'outputs': [{'quantity': 'prices',
   'sink': 'aggregate',
   'filename': '{cwd}/{default_output_dir}/MCA{Quantity}.csv'},
  {'quantity': 'capacity',
   'sink': 'aggregate',
   'filename': '{cwd}/{default_output_dir}/MCA{Quantity}.csv'}],
 'carbon_budget_control': {'budget': []},
 'global_input_files': {'projections': '{path}/input/Projections.csv',
  'global_commodities': '{path}/input/GlobalCommodities.csv'},
 'sectors': {'residential': {'type': 'default',
   'priority': 1,
   'dispatch_production': 'share',
   'technodata': '{path}/technodata/residential/Technodata.csv',
   'commodities_in': '{path}/technodata/residential/CommIn.csv',
   'commodities_out': '{path}/technodata/residential/CommOu

At this point, we can run the simulation using the commandline (`python3 -m muse
settings2.toml`). And of course, we can run it from this notebook:

In [13]:
from logging import getLogger
getLogger("muse").setLevel(100) # suppresses the log
mca = MCA.factory(model_path / "modified_settings.toml")
mca.run()
assert (Path("Results") / "Residential2020Capacity.dummy").exists()

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 149.1019498703      
0.3675729138193     0.3675729138193     0.3675729138194     0.6468967809453  0.3675729138193     897.7040172625      
0.1586603347581     0.1586603347581     0.1586603347581     0.6045622515835  0.1586603347581     1052.983832701      
0.00534123475523    0.005341234755232   0.005341234755235   0.9868065920797  0.005341234755234   219.6399956184      
0.00202447267962    0.00202447267962    0.002024472679622   0.6402354415555  0.002024472679621   184.1460808456      
0.001250110255268   0.001250110255268   0.001250110255269   0.4028712420146  0.001250110255269   243.9362311365      
0.0001937966400292  0.0001937966400293  0.0001937966400294  0.8586478166717  0.0001937966400299  307.3188749593      
3.969593492852e-07  3.969593492763e-07  3.96959349281e-07

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 24.59260538017      
0.2775083958163     0.2775083958163     0.2775083958164     0.7285952113667  0.2775083958163     148.3600126158      
0.07710650088549    0.07710650088567    0.07710650088567    0.7681349651352  0.07710650088446    1581.963825039      
0.006376477934437   0.006376477934452   0.006376477934463   0.935956970954   0.006376477934356   7709.62279059       
8.37551580473e-05   8.375515804826e-05  8.375515804892e-05  0.9915365079839  8.37551580898e-05   10909.1459966       
4.422113808713e-09  4.422113472578e-09  4.42211325731e-09   0.9999472583514  4.422112517372e-09  10972.49909201      
2.198935283418e-13  2.213623764234e-13  2.209982177521e-13  0.9999499417962  2.211057363598e-13  10972.50244333      
Optimization terminated successfully.
         Current fu

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 179.7723368976      
0.3993302006998     0.3993302006998     0.3993302006995     0.6139935096719  0.3993302006998     917.2585896334      
0.1623892560446     0.1623892560446     0.1623892560445     0.6278222636526  0.1623892560446     1085.183497054      
0.004089984236313   0.00408998423631    0.004089984236305   0.9951530431267  0.00408998423631    164.6189278119      
0.001952636177596   0.001952636177594   0.001952636177592   0.5479245906554  0.001952636177595   179.7683553218      
0.0005186625198001  0.0005186625197996  0.000518662519799   0.7589359500237  0.0005186625197996  307.4500326601      
3.416158978535e-06  3.416158978529e-06  3.416158978525e-06  0.9935399703834  3.416158978597e-06  304.9496560203      
1.709854355532e-10  1.709854384347e-10  1.70985435705e-10

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 414.631209044       
0.3829877776364     0.3829877776364     0.3829877776364     0.6299693947059  0.3829877776364     895.3338563746      
0.02403904846988    0.02403904846987    0.02403904846988    1.0              0.02403904846987    442.9393386807      
0.01125163049636    0.01125163049636    0.01125163049636    0.5621529023569  0.01125163049636    642.3568253833      
0.001904649149204   0.001904649149192   0.001904649149193   0.8494863612741  0.001904649149272   1131.740325309      
1.546976470619e-05  1.546976470611e-05  1.546976470611e-05  0.9924145494516  1.546976470659e-05  1241.817483776      
2.594798694784e-07  2.594798693995e-07  2.594798693887e-07  0.9832316901746  2.594798697893e-07  1242.859805556      
1.353023262328e-07  1.353024359135e-07  1.353024359216e-0

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 758.9496445805      
0.04019981550984    0.04019981550985    0.04019981550972    0.9661187744028  0.04019981550984    1434.913550166      
0.03295232170433    0.03295232170434    0.03295232170423    0.1941101408027  0.03295232170433    974.1654016843      
0.02705816547679    0.02705816547679    0.0270581654767     0.1982722467397  0.02705816547678    998.6154337413      
0.004706782149466   0.004706782149465   0.00470678214945    0.8397682056309  0.004706782149464   317.504540959       
0.0001545301751902  0.0001545301751902  0.0001545301751897  0.9716613542202  0.0001545301751902  16.06236933437      
1.238879921615e-05  1.238879921616e-05  1.238879921612e-05  0.9237813825879  1.238879921616e-05  4.916733972715      
2.004519216618e-08  2.004519254326e-08  2.004519255408e-0

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 569.1954516611      
0.5297453916561     0.5297453916561     0.5297453916561     0.4854204266106  0.5297453916561     1073.786536501      
0.2234707096576     0.2234707096576     0.2234707096575     0.5985101437824  0.2234707096576     1484.14884153       
0.01299984743838    0.01299984743839    0.01299984743839    1.0              0.01299984743839    511.9450201525      
0.005205556442554   0.005205556442556   0.005205556442556   0.6136849578936  0.005205556442556   321.4118501179      
0.002560653233707   0.002560653233707   0.002560653233707   0.539983536304   0.002560653233708   409.3901165727      
0.0001220033589575  0.0001220033589575  0.0001220033589575  0.9547643639     0.0001220033589575  361.1074892194      
7.841098725824e-09  7.841098754176e-09  7.841098752896e-0

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 1026.643560413      
0.05011623709528    0.05011623709529    0.05011623709533    0.9561281410911  0.05011623709529    1757.2914103        
0.04017143883108    0.04017143883108    0.04017143883111    0.2142356932608  0.04017143883108    1164.825836237      
0.01187134389619    0.01187134389619    0.0118713438962     0.7269958284032  0.01187134389619    633.8187163395      
0.0006654672589993  0.0006654672589995  0.000665467259      0.9484713219     0.0006654672589995  53.51145536036      
3.359008845864e-05  3.359008845867e-05  3.359008845869e-05  0.9498365150403  3.359008845867e-05  7.213595114132      
6.933572061441e-07  6.933572047786e-07  6.933572047751e-07  0.987739206569   6.933572047746e-07  4.045378237072      
3.947801302096e-11  3.94780458525e-11   3.947805789453e-1

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 599.4693530625      
0.06042992940962    0.0604299294096     0.06042992940964    0.9459366582215  0.06042992940959    953.8509336505      
0.02295318750859    0.02295318750859    0.0229531875086     0.6655596202526  0.02295318750858    920.6610949816      
0.001138584502195   0.001138584502195   0.001138584502196   0.9986116272963  0.001138584502195   188.3248347854      
3.838725160114e-05  3.838725160114e-05  3.838725160116e-05  0.9675657301212  3.838725160113e-05  19.4018111094       
8.316521041187e-07  8.316521040978e-07  8.316521041008e-07  0.9906520730576  8.316521041001e-07  8.914458490867      
6.362068745172e-10  6.362068753701e-10  6.36206877078e-10   0.9992350087754  6.362068771031e-10  8.833832577065      
1.40412934035e-10   1.402175213906e-10  1.402175219445e-1